# PSyKE's demo for regression tasks

Some imports.

In [1]:
from psyke import Extractor, Clustering
from psyke.tuning.pedro import PEDRO
from psyke.tuning import Objective
from psyke.tuning.crash import CRASH
from sklearn.tree import DecisionTreeRegressor
from psyke.utils.logic import pretty_theory
from psyke.utils.metrics import mae, mse, r2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from psyke.utils import Target
import pandas as pd

Import an artificial dataset.

In [2]:
#dataset = pd.read_csv("../test/resources/datasets/df.csv")
#dataset = dataset[["X", "Y", "Z4"]].dropna()
#dataset = pd.read_csv("../test/resources/datasets/CCPP.csv", sep=";", decimal=",")
dataset = pd.read_csv("../test/resources/datasets/arti.csv")

Split between train and test set in a reproducible way.

In [3]:
train, test = train_test_split(dataset, test_size=0.5, random_state=10)

scaler = StandardScaler().fit(train)
train = pd.DataFrame(scaler.transform(train), columns=train.columns, index=train.index)
test = pd.DataFrame(scaler.transform(test), columns=test.columns, index=test.index)

normalization = {key: (m, s) for key, m, s in zip(train.columns, scaler.mean_, scaler.scale_)}

We select and train a predictor.

In [4]:
#predictor = KNeighborsRegressor(n_neighbors=3).fit(train.iloc[:, :-1], train.iloc[:, -1])
predictor = DecisionTreeRegressor().fit(train.iloc[:, :-1], train.iloc[:, -1])
#predictor = LinearRegression().fit(train.iloc[:, :-1], train.iloc[:, -1])

m, s = normalization[test.columns[-1]]

predicted = predictor.predict(test.iloc[:, :-1]).flatten() * s + m
true = test.iloc[:, -1] * s + m

print(f'MAE = {mae(true, predicted):.2f}')
print(f'MSE = {mse(true, predicted):.2f}')
print(f'R2 = {r2(true, predicted):.2f}')

MAE = 0.00
MSE = 0.00
R2 = 1.00


We define a function to print the extractors’ evaluation

In [5]:
def evaluate(name, extractor, true, predicted):
    extracted = extractor.unscale(extractor.predict(test.iloc[:, :-1]), test.columns[-1])
    print(f'{name} performance ({extractor.n_rules} rules):\n'
          f'MAE = {mae(true, extracted):.2f}\nMAE fidelity = {mae(predicted, extracted):.2f}\n'
          f'R2 = {r2(true, extracted):.2f}\nR2 fidelity = {r2(predicted, extracted):.2f}\n')

We create several extractors that use ITER, GridEx and GridREx algorithms to extract prolog rules from the predictor.

In [6]:
it = Extractor.iter(predictor, min_update=1.0 / 10, n_points=1, max_iterations=600,
                    min_examples=100, threshold=.4, normalization=normalization)
theory_from_iter = it.extract(train)
evaluate('ITER', it, true, predicted)
print('ITER extracted rules:\n\n' + pretty_theory(theory_from_iter))

ITER performance (4 rules):
MAE = 0.55
MAE fidelity = 0.55
R2 = -6.34
R2 fidelity = -6.34

ITER extracted rules:

z(X, Y, 1.11) :-
    X in [-1.70, -0.00], Y in [-1.69, 0.22].
z(X, Y, 0.19) :-
    X in [-1.70, -0.00], Y in [0.22, 1.75].
z(X, Y, -0.11) :-
    X in [-0.00, 1.75], Y in [-1.69, 0.22].
z(X, Y, -1.29) :-
    X in [-0.00, 1.75], Y in [0.22, 1.75].


In [7]:
creepy = Extractor.creepy(predictor, depth=3, error_threshold=0.02, output=Target.CONSTANT,
                          normalization=normalization, clustering=Clustering.exact)
theory_from_creepy = creepy.extract(train)
evaluate('CReEPy', creepy, true, predicted)
print('CReEPy extracted rules:\n\n' + pretty_theory(theory_from_creepy))

CReEPy performance (3 rules):
MAE = 0.08
MAE fidelity = 0.08
R2 = 0.81
R2 fidelity = 0.81

CReEPy extracted rules:

z(X, Y, 0.7) :-
    X in [0.00, 0.49], Y in [0.00, 0.49].
z(X, Y, 0.4) :-
    X in [0.00, 0.49], Y in [0.00, 0.99].
z(X, Y, 0.145238).


In [8]:
creepy = Extractor.creepy(predictor, depth=3, error_threshold=0.02, output=Target.REGRESSION,
                          normalization=normalization, clustering=Clustering.exact)
theory_from_creepy = creepy.extract(train)
evaluate('CReEPy', creepy, true, predicted)
print('CReEPy extracted rules:\n\n' + pretty_theory(theory_from_creepy))

CReEPy performance (3 rules):
MAE = 0.03
MAE fidelity = 0.03
R2 = 0.94
R2 fidelity = 0.94

CReEPy extracted rules:

z(X, Y, Z) :-
    X in [0.00, 0.49], Y in [0.00, 0.49], Z is 0.7.
z(X, Y, Z) :-
    X in [0.00, 0.49], Y in [0.00, 0.99], Z is 0.4.
z(X, Y, Z) :-
    Y in [0.00, 0.99], Z is 0.38 - 0.09 * X - 1.73 * Y.


In [9]:
creepy = Extractor.creepy(predictor, depth=2, error_threshold=0.02, output=Target.REGRESSION,
                          normalization=normalization, clustering=Clustering.cream)
theory_from_creepy = creepy.extract(train)
evaluate('CReEPy', creepy, true, predicted)
print('CReEPy extracted rules:\n\n' + pretty_theory(theory_from_creepy))

CReEPy performance (4 rules):
MAE = 0.01
MAE fidelity = 0.01
R2 = 0.94
R2 fidelity = 0.94

CReEPy extracted rules:

z(X, Y, Z) :-
    X in [0.51, 0.99], Y in [0.00, 0.49], Z is 0.3.
z(X, Y, Z) :-
    X in [0.50, 0.99], Y in [0.00, 0.98], Z is 0.0.
z(X, Y, Z) :-
    X in [0.00, 0.49], Y in [0.50, 0.99], Z is 0.4.
z(X, Y, Z) :-
    Y in [0.00, 0.99], Z is 0.7.


In [10]:
crash = CRASH(predictor, train, max_depth=3, patience=1, readability_tradeoff=.5,
              algorithm=CRASH.Algorithm.ExACT, output=Target.REGRESSION, normalization=normalization)
crash.search()
(_, _, depth, threshold) = crash.get_best()[0]

creepy = Extractor.creepy(predictor, depth=depth, error_threshold=threshold, output=Target.REGRESSION,
                          clustering=Clustering.exact)
theory_from_creepy = creepy.extract(train)
evaluate('CReEPy', creepy, true, predicted)
print('CReEPy extracted rules:\n\n' + pretty_theory(theory_from_creepy))

Algorithm.ExACT. Depth: 1. Threshold = 0.00. MAE = 0.06, 2 rules
Algorithm.ExACT. Depth: 1. Threshold = 0.00. MAE = 0.06, 2 rules

Algorithm.ExACT. Depth: 2. Threshold = 0.00. MAE = 0.06, 2 rules
Algorithm.ExACT. Depth: 2. Threshold = 0.00. MAE = 0.06, 2 rules

**********************
Best Algorithm.ExACT
**********************
MAE = 0.06, 2 rules
Threshold = 0.00
Depth = 2

**********************
Best   MAE  
**********************
MAE = 0.06, 2 rules
Threshold = 0.00
Depth = 2

**********************
Best N rules
**********************
MAE = 0.06, 2 rules
Threshold = 0.00
Depth = 2

CReEPy performance (3 rules):
MAE = 0.69
MAE fidelity = 0.69
R2 = -8.82
R2 fidelity = -8.82

CReEPy extracted rules:

z(X, Y, Z) :-
    X in [-1.70, 0.01], Y in [-1.69, 0.02], Z is 1.37.
z(X, Y, Z) :-
    X in [-1.70, 0.01], Y in [-1.69, 1.75], Z is 0.19.
z(X, Y, Z) :-
    Y in [-1.69, 1.75], Z is -0.76 - 0.02 * X - 0.50 * Y.


In [11]:
crash = CRASH(predictor, train, max_depth=3, patience=1, readability_tradeoff=.75, algorithm=CRASH.Algorithm.CREAM,
              normalization=normalization)
crash.search()
(_, _, depth, threshold) = crash.get_best()[0]

creepy = Extractor.creepy(predictor, depth=depth, error_threshold=threshold, output=Target.REGRESSION,
                          clustering=Clustering.cream)
theory_from_creepy = creepy.extract(train)
evaluate('CReEPy', creepy, true, predicted)
print('CReEPy extracted rules:\n\n' + pretty_theory(theory_from_creepy))

Algorithm.CREAM. Depth: 1. Threshold = 0.00. MAE = 0.12, 2 rules
Algorithm.CREAM. Depth: 1. Threshold = 0.00. MAE = 0.12, 2 rules

Algorithm.CREAM. Depth: 2. Threshold = 0.00. MAE = 0.02, 3 rules
Algorithm.CREAM. Depth: 2. Threshold = 0.00. MAE = 0.02, 3 rules

**********************
Best Algorithm.CREAM
**********************
MAE = 0.02, 3 rules
Threshold = 0.00
Depth = 2

**********************
Best   MAE  
**********************
MAE = 0.02, 3 rules
Threshold = 0.00
Depth = 2

**********************
Best N rules
**********************
MAE = 0.12, 2 rules
Threshold = 0.00
Depth = 1

CReEPy performance (4 rules):
MAE = 0.69
MAE fidelity = 0.69
R2 = -9.40
R2 fidelity = -9.40

CReEPy extracted rules:

z(X, Y, Z) :-
    X in [0.07, 1.75], Y in [-1.69, 0.00], Z is -0.19.
z(X, Y, Z) :-
    X in [0.02, 1.75], Y in [-1.69, 1.71], Z is -1.37.
z(X, Y, Z) :-
    X in [-1.69, 0.01], Y in [0.03, 1.75], Z is 0.19.
z(X, Y, Z) :-
    Y in [-1.69, 1.75], Z is 1.37.


In [12]:
pedro = PEDRO(predictor, train, max_mae_increase=1.2, min_rule_decrease=0.9, readability_tradeoff=0.1,
              max_depth=1, patience=1, algorithm=PEDRO.Algorithm.GRIDEX, objective=Objective.MODEL,
              normalization=normalization)
pedro.search()
(_, _, threshold, grid) = pedro.get_best()[0]

gridEx = Extractor.gridex(predictor, grid, threshold=threshold, normalization=normalization)
theory_from_gridEx = gridEx.extract(train)
evaluate('GridEx', gridEx, true, predicted)
print('GridEx extracted rules:\n\n' + pretty_theory(theory_from_gridEx))

Algorithm.GRIDEX. Grid (1). Fixed (2). Threshold = 0.00. MAE = 0.00, 4 rules
Algorithm.GRIDEX. Grid (1). Fixed (2). Threshold = 0.00. MAE = 0.00, 8 rules

Algorithm.GRIDEX. Grid (1). Fixed (3). Threshold = 0.00. MAE = 0.00, 17 rules
Algorithm.GRIDEX. Grid (1). Fixed (3). Threshold = 0.00. MAE = 0.00, 26 rules

Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 2)]). Threshold = 0.00. MAE = 0.00, 28 rules
Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 2)]). Threshold = 0.00. MAE = 0.00, 30 rules

Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 3)]). Threshold = 0.00. MAE = 0.00, 33 rules
Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 3)]). Threshold = 0.00. MAE = 0.00, 36 rules

Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 5)]). Threshold = 0.00. MAE = 0.00, 41 rules
Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 5)]). Threshold = 0.00. MAE = 0.00, 46 rules

Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 10)]). Threshold = 0.00. MAE = 0.00, 56 rules
Algorithm.GRIDEX. Grid (1). Adaptive ([(0.99, 1

In [13]:
#pedro = PEDRO(predictor, train, max_mae_increase=1.2, min_rule_decrease=0.9, readability_tradeoff=0.1,
#              max_depth=2, patience=1, algorithm=PEDRO.Algorithm.GRIDREX, objective=Objective.MODEL)
#pedro.search()
(_, _, threshold, grid) = pedro.get_best()[0]

gridREx = Extractor.gridrex(predictor, grid, threshold=threshold, normalization=normalization)
theory_from_gridREx = gridREx.extract(train)
evaluate('GridREx', gridREx, true, predicted)
print('GridREx extracted rules:\n\n' + pretty_theory(theory_from_gridREx))

**********************
Best Algorithm.GRIDEX
**********************
MAE = 0.00, 4 rules
Threshold = 0.00
Iterations = 1
Strategy = Fixed (2)

**********************
Best   MAE  
**********************
MAE = 0.00, 72 rules
Threshold = 0.00
Iterations = 1
Strategy = Adaptive ([(0.33, 2), (0.67, 3)])

**********************
Best N rules
**********************
MAE = 0.00, 4 rules
Threshold = 0.00
Iterations = 1
Strategy = Fixed (2)

GridREx performance (86 rules):
MAE = 0.00
MAE fidelity = 0.00
R2 = 0.99
R2 fidelity = 0.99

GridREx extracted rules:

z(X, Y, 0.7) :-
    X in [0.00, 0.50], Y in [0.00, 0.49].
z(X, Y, 0.39) :-
    X in [0.00, 0.50], Y in [0.49, 0.99].
z(X, Y, 0.3) :-
    X in [0.50, 0.99], Y in [0.00, 0.49].
z(X, Y, 0.0) :-
    X in [0.50, 0.99], Y in [0.49, 0.99].
z(X, Y, 0.7) :-
    X in [0.00, 0.50], Y in [0.00, 0.49].
z(X, Y, 0.39) :-
    X in [0.00, 0.50], Y in [0.49, 0.99].
z(X, Y, 0.3) :-
    X in [0.50, 0.99], Y in [0.00, 0.49].
z(X, Y, 0.0) :-
    X in [0.50, 0.99], Y

In [14]:
cart = Extractor.cart(predictor, max_depth=4, max_leaves=4, simplify=True, normalization=normalization)
theory_from_cart = cart.extract(train)
evaluate('CART', cart, true, predicted)
print('CART extracted rules:\n\n' + pretty_theory(theory_from_cart))

CART performance (4 rules):
MAE = 0.00
MAE fidelity = 0.00
R2 = 1.00
R2 fidelity = 1.00

CART extracted rules:

z(X, Y, 0.3) :-
    X > 0.50, Y =< 0.49.
z(X, Y, 0.0) :-
    X > 0.50, Y > 0.49.
z(X, Y, 0.7) :-
    Y =< 0.50.
z(X, Y, 0.4).
